In [ ]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib
import time
import psutil
import os

In [ ]:
# TFLite 모델 및 스케일러 로드
interpreter = tflite.Interpreter(model_path="./model/cnn1d_model_compatible.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load('./model/scaler_cnn.joblib')

In [ ]:
# Test 데이터셋 불러오기
column_names = ['timestamp', 'voltage', 'current', 'label']
test_df = pd.read_csv('../../data/test/test_dataset_187.csv', names=column_names, header=None)
test_df = test_df.dropna(subset=['voltage', 'current']).reset_index(drop=True)

In [ ]:
# 파생 피처 생성 함수
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
# 슬라이딩 윈도우 기반 피처 생성
def generate_sequence_features(df, window_size=8):
    features = []
    labels = []

    for i in range(window_size, len(df)):
        voltage_seq = df['voltage'].iloc[i - window_size:i].values
        current_seq = df['current'].iloc[i - window_size:i].values
        feat = add_features(voltage_seq, current_seq)
        features.append(feat)
        labels.append(df['label'].iloc[i])  # 마지막 인덱스의 라벨 사용

    return np.array(features), np.array(labels)


In [ ]:
# 피처 및 라벨 생성
window_size = 8
X_test, y_test = generate_sequence_features(test_df, window_size=window_size)

# 정규화
X_test_scaled = scaler.transform(X_test)

# CNN 입력 형태로 reshape: (samples, timesteps, features)
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], window_size, X_test_scaled.shape[1]))

In [ ]:
# 예측 함수
def predict_with_tflite(interpreter, X):
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']
    predictions = []

    for sample in X:
        sample = sample.astype(np.float32)
        interpreter.set_tensor(input_index, np.expand_dims(sample, axis=0))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        predictions.append(output_data[0][0])

    return np.array(predictions)

In [ ]:
# 예측
y_pred_prob = predict_with_tflite(interpreter, X_test_scaled)
y_pred = (y_pred_prob > 0.48).astype(int)

In [ ]:
# 후처리 함수
def postprocess_anomalies(y_pred_bin, min_consecutive=1):
    y_post = np.zeros_like(y_pred_bin)
    count = 0
    for i, val in enumerate(y_pred_bin):
        if val == 1:
            count += 1
        else:
            if count >= min_consecutive:
                y_post[i - count:i] = 1
            count = 0
    if count >= min_consecutive:
        y_post[len(y_pred_bin)-count:] = 1
    return y_post

# 후처리 적용
y_pred_post = postprocess_anomalies(y_pred, min_consecutive=1)

In [ ]:
# 분류 리포트 출력
print("📋 (후처리 적용) 분류 리포트:\n")
print(classification_report(y_test, y_pred_post, target_names=['정상', '아크']))

In [ ]:
# 감지 지연 시간 계산 함수
def calculate_detection_delay(y_pred_bin, label, sampling_interval=0.001163):
    try:
        # 아크의 시작 인덱스 찾기
        arc_start = np.where(label == 1)[0][0]
        # 예측 값에서 아크를 처음 발견한 인덱스 찾기
        detected = np.where(y_pred_bin[arc_start:] == 1)[0]
        if len(detected) == 0:
            print("⚠️ 아크를 탐지하지 못했습니다.")
            return None
        detect_index = detected[0] + arc_start
        delay = detect_index - arc_start
        time_detected = detect_index * sampling_interval
        print(f"✅ 아크 시작 인덱스: {arc_start}")
        print(f"✅ 모델 이상 감지 인덱스: {detect_index}")
        print(f"⏱️ 감지 지연 시간: {delay} 샘플")
        print(f"⏱️ 감지된 시점 (초): {time_detected:.6f} 초")
        return arc_start, detect_index, delay
    except IndexError:
        print("⚠️ 라벨 데이터에 아크가 포함되어 있지 않습니다.")
        return None
    
# 감지 지연 시간 계산
result = calculate_detection_delay(y_pred_post, y_test)
if result is not None:
    arc_start, detect_index, delay = result
else:
    arc_start, detect_index, delay = None, None, None

In [ ]:
# 전체 예측 시간
start_total = time.perf_counter()
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob.flatten() > 0.5).astype(int)
elapsed_total = time.perf_counter() - start_total
print(f"⏱️ 전체 예측 소요 시간: {elapsed_total:.10f}초")

# 후처리 + 첫 이상 감지까지 시간
start_first = time.perf_counter()
y_pred_post = postprocess_anomalies(y_pred, min_consecutive=8)
first_index = np.where(y_pred_post == 1)[0]
elapsed_first = time.perf_counter() - start_first

if len(first_index) > 0:
    print(f"🟡 첫 번째 이상 탐지 인덱스: {first_index[0]}")
    print(f"⏱️ 첫 이상 탐지까지 걸린 시간: {elapsed_first:.10f}초")
else:
    print("⚠️ 후처리된 이상 탐지 결과가 없습니다.")


In [ ]:
# 모듈 임포트
import psutil
import os

# 리소스 사용량 출력 함수
def print_resource_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    cpu_percent = process.cpu_percent(interval=1.0)
    print(f"🧠 메모리 사용량: {mem_info.rss / 1024 ** 2:.2f} MB")
    print(f"🧮 CPU 사용률: {cpu_percent:.2f}%")


# 전체 예측 시간 측정 + 시스템 리소스 출력
print("🔍 예측 전 시스템 상태:")
print_resource_usage()

start_total = time.perf_counter()
y_pred = model.predict(X_test_scaled)
elapsed_total = time.perf_counter() - start_total

print("🔍 예측 후 시스템 상태:")
print_resource_usage()

print(f"⏱️ 전체 예측 소요 시간: {elapsed_total:.10f}초")



---
# 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# 한글 폰트 설정 (예: 맑은 고딕)
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 사용자
# plt.rcParams['font.family'] = 'AppleGothic'   # macOS 사용자
# plt.rcParams['font.family'] = 'NanumGothic'   # Linux 사용자 (Nanum 폰트 설치 필요)

# 마이너스 부호 깨짐 방지
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
def plot_predictions(y_pred_bin, arc_start=None, detect_index=None):
    plt.figure(figsize=(15, 4))
    plt.plot(y_pred_bin, label='예측 이상 (1=이상)', color='red', linewidth=1)

    if arc_start is not None:
        plt.axvline(x=arc_start, color='blue', linestyle='--', label='실제 아크 시작')
    if detect_index is not None:
        plt.axvline(x=detect_index, color='green', linestyle='--', label='감지된 시점')

    plt.title("CNN 이상 탐지 결과")
    plt.xlabel("샘플 인덱스")
    plt.ylabel("예측값")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_predictions(y_pred_post, arc_start, detect_index)


In [ ]:
def plot_current_waveform(test_df):
    plt.figure(figsize=(15, 4))
    plt.plot(test_df['current'], label='전류 파형', alpha=0.7)
    plt.title("전체 전류 파형")
    plt.xlabel("샘플 인덱스")
    plt.ylabel("전류 (A)")
    plt.grid(True)
    plt.tight_layout()
    plt.legend()
    plt.show()

plot_current_waveform(test_df)

In [ ]:
def compare_postprocessing(y_pred_bin, y_pred_post, title="후처리 전후 비교"):
    plt.figure(figsize=(15, 4))
    plt.plot(y_pred_bin, label='후처리 전', color='orange', alpha=0.6)
    plt.plot(y_pred_post, label='후처리 후', color='green', alpha=0.6)
    plt.title(title)
    plt.xlabel("샘플 인덱스")
    plt.ylabel("이상 탐지 결과")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

compare_postprocessing(y_pred, y_pred_post)




In [ ]:
def compare_prediction_vs_label(y_true, y_pred_post):
    plt.figure(figsize=(15, 4))
    plt.plot(y_true, label='실제 라벨', alpha=0.5)
    plt.plot(y_pred_post, label='모델 예측', alpha=0.7)
    plt.title("실제 라벨 vs 모델 예측 비교")
    plt.xlabel("샘플 인덱스")
    plt.ylabel("값 (0=정상, 1=아크)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

compare_prediction_vs_label(y_true_post, y_pred_post)

# # 차이점 위치 찾기
# mismatch_indices = np.where(y_true_post != y_pred_post)[0]
# print("불일치 인덱스:", mismatch_indices)
# print("불일치 개수:", len(mismatch_indices))